In [ ]:
import numpy as np
from communication_system import *
from deep_receiver import *
import pickle

In [ ]:
# some data about this experiment and the models' parameters
var_file = "model/additional_noise_deep_set.data"
retrain = False

moder = {
    "bpsk": PSKModem(2),
    "qpsk": PSKModem(4)
}

ham74 = Hamming74()

class DeepSet:
    def __init__(self, mod_name, noise):
        self.mod = moder[mod_name]
        self.mod_name = mod_name
        self.noise = noise
        self.net = Conv_DenseNet_1D()
        self.train_ebr_log = None

class NoiseTestLog:
    def __init__(self, mod_name, noise):
        self.mod_name = mod_name
        self.noise = noise
        self.new_model_ebr_log = []
        self.old_model_ebr_log = []

In [ ]:
# test1：AWGN channel
# generate training set 
train_set_sample_num =  2 * 1000 * 9
data_bits = gen_rand_code(32, train_set_sample_num)
deep_set = None


# train the model or load the model from the files.

if not os.path.exists(var_file) or retrain:
    print("need to train nets")
    deep_set = [
        DeepSet("bpsk", 'awgn'),
        DeepSet("qpsk",'awgn'),
        DeepSet("bpsk", 'aggn'),
        DeepSet("qpsk", 'aggn')
    ]
    
    
    coded_data_bits = ham74.encode(data_bits)
    EbN0 = np.arange(9)
    for setting in deep_set:
        mod_data, _, _ = modulate(coded_data_bits, setting.mod, True)
        
        # generate noise
        SNR = EbN02SNR(EbN0, setting.mod.num_bits_symbol, 8)
        
        # generate training set
        noised_data = []
        for mod_data_part, snr in zip(mod_data.reshape([9, len(mod_data)//9, -1]), SNR):
            noised_data.append(add_noise(mod_data_part, setting.noise, snr))
        noised_data = np.concatenate(noised_data, 0)
        train_set = (torch.tensor(noised_data), torch.tensor(data_bits))
        
        # trian the model
        print(f"net(modulate:{setting.mod}, noise:{setting.noise}) start training")
        setting.train_ebr_log = train(
            net = setting.net,
            data_set = train_set,
            batch_size = 256,
            lr = 0.001,
            lr_gain = 0.1,
            epochs = 8,
            steady_epochs = 2,
            momentum = 0.9
        )
        print(f"net(modulate:{setting.mod}, noise:{setting.noise}) training done")
        
    # store the parameter of the trained model. 
    print(f"save the data to the file:{var_file}")
    fp = open(var_file, "wb")
    pickle.dump(deep_set, fp)
    fp.close()
else:
    print(f"load the net param from the file:{var_file}")
    fp = open(var_file, "rb")
    deep_set = pickle.load(fp)
    fp.close()


In [ ]:
# model is ready, then test it

test_set_sample_num =  2 * 1000 * 19
data_bits = gen_rand_code(32, train_set_sample_num)

coded_data_bits = ham74.encode(data_bits)
EbN0 = np.linspace(0, 8, 19)

noise_test_logs = []

for setting in deep_set:
    mod_data, _, _ = modulate(coded_data_bits, setting.mod, True)

    # generate noise
    SNR = EbN02SNR(EbN0, setting.mod.num_bits_symbol, 8)

    # generate testing dataset for every SNR and test the model respectively
    noise_test_log = NoiseTestLog(setting.mod_name, setting.noise)
    for mod_data_part, snr in zip(mod_data.reshape([len(SNR), len(mod_data)//len(SNR), -1]), SNR):
        noised_data = add_noise(mod_data_part, setting.noise, snr)
        test_set = (torch.tensor(noised_data), torch.tensor(data_bits))
        # test the deep_receiver
        deep_net_ebr = run_epoch(setting.net, test_set, 256)
        noise_test_log.new_model_ebr_log.append(deep_net_ebr)
        # test the traditional receiver
        out_put_data = old_receiver(noised_data, setting.mod, ham74)
        old_model_ebr = (out_put_data != data_bits).mean()
        noise_test_log.old_model_ebr_log.append(old_model_ebr)
    noise_test_logs.append(noise_test_log)

commpy.modulation.PSKModem